<h3> Extracting protein sequences' features using ProtBert pretrained-model <h3>

<b>1. Load necessry libraries including huggingface transformers<b>

In [5]:
!pip install -q transformers

In [6]:
import tensorflow as tf
from transformers import TFBertModel, BertTokenizer,BertConfig
import re
import numpy as np

<b>2. Load the vocabulary and ProtBert Model</b>

In [7]:
tokenizer = BertTokenizer.from_pretrained("Rostlab/prot_bert", do_lower_case=False )

In [8]:
model = TFBertModel.from_pretrained("Rostlab/prot_bert", from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint

<b>3. Create or load sequences and map rarely occured amino acids (U,Z,O,B) to (X)<b>

In [28]:
# sequences_Example = ["A E T C Z A O","MSYTDNPPQTKRALSLDDLVNHDENEKVKLQKLSEAANGSRPFAENLESDINQTETGQAAPIDNYKESTGHGSHSQKPKSRKSSNDDEETDTDDEMGASGEINFDSEMDFDYDKQHRNLLSNGSPPMNDGSDANAKLEKPSDDSIHQNSKSDEEQRIPKQGNEGNIASNYITQVPLQKQKQTEKKIAGNAVGSVVKKEEEANAAVDNIFEEKATLQSKKNNIKRDLEVLNEISASSKPSKYKNVPIWAQKWKPTIKALQSINVKDLKIDPSFLNIIPDDDLTKSVQDWVYATIYSIAPELRSFIELEMKFGVIIDAKGPDRVNPPVSSQCVFTELDAHLTPNIDASLFKELSKYIRGISEVTENTGKFSIIESQTRDSVYRVGLSTQRPRFLRMSTDIKTGRVGQFIEKRHVAQLLLYSPKDSYDVKISLNLELPVPDNDPPEKYKSQSPISERTKDRVSYIHNDSCTRIDITKVENHNQNSKSRQSETTHEVELEINTPALLNAFDNITNDSKEYASLIRTFLNNGTIIRRKLSSLSYEIFEGSKKVM"]
# sequences_Example = ["A E T C Z A O","MSYTDNPPQTKRALSLDDLVNHDENEKVKLQKLSEAANGSRPFAENLESDINQTETGQAAPIDNYKESTGHGSHSQKPKSRKSSNDDEETDTDDEMGASGEINFDSEMDFDYDKQHRNLLSNGSPPMNDGSDANAKLEKPSDDSIHQNSKSDEEQRIPKQGNEGNIASNYITQVPLQKQKQTEKKIAGNAVGSVVKKEEEANAAVDNIFEEKATLQSKKNNIKRDLEVLNEISASSKPSKYKNVPIWAQKWKPTIKALQSINVKDLKIDPSFLNIIPDDDLTKSVQDWVYATIYSIAPELRSFIELEMKFGVIIDAKGPDRVNPPVSSQCVFTELDAHLTPNIDASLFKELSKYIRGISEVTENTGKFSIIESQTRDSVYRVGLSTQRPRFLRMSTDIKTGRVGQFIEKRHVAQLLLYSPKDSYDVKISLNLELPVPDNDPPEKYKSQSPISERTKDRVSYIHNDSCTRIDITKVENHNQNSKSRQSETTHEVELEINTPALLNAFDNITNDSKEYASLIRTFLNNGTIIRRKLSSLSYEIFEGSKKVM"]
with open('protein_preprocessed.txt') as file:
    sequences_Example = file.readlines()
    sequences_Example = [line.rstrip() for line in lines]

In [29]:
sequences_Example = [re.sub(r"[UZOB]", "X", sequence) for sequence in sequences_Example]

<b>4. Tokenize, encode sequences and load it into the GPU if possibile<b>

In [30]:
ids = tokenizer.batch_encode_plus(sequences_Example, add_special_tokens=True, padding=True, return_tensors="tf")

In [32]:
input_ids = ids['input_ids']
attention_mask = ids['attention_mask']

<b>5. Extracting sequences' features and load it into the CPU if needed<b>

In [33]:
embedding = model(input_ids)[0]

In [34]:
embedding = np.asarray(embedding)

In [36]:
attention_mask = np.asarray(attention_mask)

<b>6. Remove padding ([PAD]) and special tokens ([CLS],[SEP]) that is added by ProtBert model<b>

In [37]:
features = [] 
for seq_num in range(len(embedding)):
    seq_len = (attention_mask[seq_num] == 1).sum()
    seq_emd = embedding[seq_num][1:seq_len-1]
    features.append(seq_emd)

In [38]:
features[0].shape

array([ 0.09233966,  0.13914385, -0.05241557, ..., -0.13947985,
       -0.04280062,  0.07431392], dtype=float32)